$$\require{mhchem}$$
# Monod-type denitrification model

In this notebook I use the Monod equations proposed by Störiko et al. 2021 and 2022.

Reactions (Störiko et al 2021):

1. *denitrification 1*: $\ce{7NO3- + C4H6O4 ->[nar] 7NO2- + 4CO2 + 3H2O}$
2. *denitrificantion 2*: $\ce{14NO2- + 14H+ + 3C4H6O4 ->[nir] 7N2 + 12CO2 + 16H2O}$
3. *aerobic respiration*: $\ce{7O2 + 2C4H6O4 -> 7NO2- + 8CO2 + 6H2O}$
---------------
In this model, denitrification rates are described by a Monod term:

$r_{j} = \nu_{max}^i B \frac{C_{j}}{C_{j} + K_{j}} \frac{I_{reac}^i}{I_{reac}^i + C_{O2}} $

for $j \in (nar, nir)$

the aerobic respiration is also described by another monod term:

$r_{\ce{O2}} = \nu_{\ce{O2}}^i B \frac{C_{\ce{O2}}}{C_{\ce{O2}} + K_{\ce{O2}}} \frac{C_{\ce{DOC}}}{C_{\ce{DOC}} + K_{\ce{DOC}}} $  

--------------------
DOC consumption is modelled by adding the stoichiometric corrected rates from the denitrification and aerobic oxidation rates:

$r_{\ce{DOC}}^{j} = \frac{\gamma_{\ce{DOC}}^{j}}{\gamma_{\ce{A}}^{j}}*r_{j}$

for $j \in (nar, nir, \ce{O2})$

and the full rate is defined by the expression:

$r_{DOC} = \sum_j r_{DOC}^j$

DOC release from the sediment contained POC is modelled with the expression:

$r_{rel} = k_{rel}^{DOC}*(C_{DOC}^{sat}- C_{DOC}) $

The final expression for change in $\ce{DOC}$ is then:

$\frac{d[DOC]}{dt} = r_{rel} - r_{DOC} $

---------------------

Bacterial growth is represented by the following reaction:

$\ce{3C4H6O4 + 2NH4+ -> C10H18O5N2 + 2CO2 + 3H2O + 2H+}$

$\ce{NH4+}$ is assumed non limiting. Thus biological growth is modelled with the following expression:

$r_{growth}^{i} = Y_i*r_{DOC}^i$ 

And the specific yield of each consumption reaction is modelled by a logistic function assuming a maximum bacterial growth:

$Y_i = Y_{i,max}*\left(1-\frac{B}{B_{max}}\right)$

Bacterial decay modelled with first order rate:

$r_{dec} = k_{dec}*B$

The final expression for bacterial density change is:

$ \frac{dB}{dt} = \sum_i r_{growth}^{i} - r_{dec}$

#### importing libraries

In [1]:
import os
import numpy as np
import scipy
import sunode
import matplotlib

## Define the RHS


In [2]:
def rhs(t, y, p):
    # Enzime rates:
    r_nar = p.nu_no3*y.B*y.no3/(y.no3+p.Kno3)*(p.Ino3/(p.Ino3+y.o2))
    r_nir = p.nu_no2*y.B*y.no2/(y.no2+p.Kno2)*(p.Ino2/(p.Ino2+y.o2))
    # Nitrate and nitrite rates:
    dno3dt = -r_nar
    dno2dt = r_nar-r_nir
    #O2 rate
    do2dt = -p.nu_o2*y.B*y.o2/(y.o2+p.Ko2)*(y.doc/(y.doc+p.Kdoc))
    #DOC rate
    ddocdt = -(1/7)*r_nar+(-(3/14)*r_nir)+(-(2/7)*do2dt)+p.k_rel*(p.c_sat-y.doc)
    # B rate:
    Y_nar = p.Y_narmax*(1-y.B/p.Bmax) 
    Y_nir = p.Y_nirmax*(1-y.B/p.Bmax)
    Y_o2 = p.Y_o2max*(1-y.B/p.Bmax)
    dBdt = (1/7)*r_nar*Y_nar + (3/14)*r_nir*Y_nir + (2/7)*do2dt*Y_o2 -p.k_dec*y.B
    return np.array([dno3dt,dno2dt,do2dt,ddocdt,dBdt]) #rhs form
    

# Define the Transport Model

c:\users\vcant\onedrive\kassel\secunda\src\external\nitrogene\nitrogene\reaction_model.py:82: RuntimeWarning: divide by zero encountered in log
  d[key]["log"] = np.log(d[key]["val"])


## Prepare the measurement data
We read in the experimental data from the batch experiment of Qu et al. (2015) and process it for later use in the likelihood:
We apply a Box-Cox transform to the data to improve homoscedasticity of the residuals. Transformation parameters are chosen based on the analysis in error_model.ipynb.



## Set up the statistical model in PyMC3
Defining our statistical model consists of three steps:
1. Define the prior distribution for all parameters that we want to estimate.
2. Compute the simulation result by solving the ODE.
3. Define the likelihood  based on the simulation result and our error model.

The first step also involves computing the actual model parameters from the parameters that will be estimated. We define all priors for the log parameters to account for variability/uncertianty over several orders of magnitude and to ensure positivity. For most parameters the transformation, thus, simply consists in applying an exponential function.
Furthermore, we compute $r_{trsc}^{max}$ and $k_{trsl}$ based on the mRNA and enzyme half-lives, $T_{intra}^{max}$ and $E_{intra}^{max}$ (in ``model_builder.transform_parameters``).
We also use a reparametrization for the Monod parameters of the aerobic respiration rate to avoid strong correlations in the posterior.
Instead of choosing the rate at the limit of $C_{O2} \to \infty$, $r_{max}^{O2}$, as a parameter we choose the rate $r_{C_{fix}}^{O2}$ at a fixed oxygen concentration $C_{fix}^{O2}$ as a parameter.
$r_{max}^{O2}$ can then be recalculated:
$$r_{max}^{O2} = r_{C_{fix}^{O2}} \cdot \frac{K_s^{O2} + C_{fix}^{O2}}{C_{fix}^{O2}} $$
We do not have literature values for $r_{C_{fix}}^{O2}$ to choose a prior.
We therefore approximate the prior's mean and standard deviation based on samples from the original priors.

The likelihood is a Student-T distirbution that take the Box-Cox transformed simulation results as mean. For nitrite and oxygen, we add a constant background value to the concentrations before the transform. 
Variances consist of the measurement variances from triplicate measurements and an estimated constant term that accoutns for model errors.


## Sample the prior

We create a function that computes the log probability value and its gradient with respect to parameters and apply it to all prior points.

Now we check if there are NaN values in the logp or its gradient. This can happen for extreme parameter combinations where the ODE becomes very difficult to solve numerically and the ODE solver fails.



## Check gradients and stability numerically
Let's time the evaluation of the logp function and the gradients for 3 prior points.